# Accurolab

This is the Accurolab Haystack pipeline (derived from Tutorial 1 and 3 from Haystack)

In [ ]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.document_store.memory import InMemoryDocumentStore


import pandas as pd
import io
import re

import os

### Instatiate Document store

In [ ]:
document_store = InMemoryDocumentStore()

print(document_store)

## Preprocessing of documents

- Download documents from Google folder (Option 1)
- Reading documents directly from local folder (Option "-

### From Google drive (To be developed)

In [ ]:
google_url = "https://drive.google.com/drive/folders/1a2T_wwoVHLZpAsd0-3lLziQ145TJyoX1?usp=sharing" 

In [ ]:
# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
#doc_dir = "data/article_txt_got"
# doc_dir = "data/"
# fetch_archive_from_http(url=google_url, output_dir=doc_dir)


### From local folder

In [ ]:
PROJECT_DIRECTORY = os.getcwd()
ARTICLES_FOLDER = PROJECT_DIRECTORY+"/data/CKB-articles-scrape/"

In [ ]:
import pathlib

# define the path
path_folder = pathlib.Path(ARTICLES_FOLDER)

# # define the pattern
json_files = "*.json"

articles = [ json_file.absolute().as_posix() for json_file in path_folder.glob(json_files)]
articles

In [ ]:
dicts_textContent = []

for article in articles:
    try:
        df_article = pd.read_json(article)
    except:
        print(article)
        continue
    complete_text = df_article['textContent'] + df_article['excerpt'] + df_article['title']
    list_textContent = complete_text.tolist() 

    for i in range(len(list_textContent)):
        dicts_textContent.append({'text': df_article['textContent'][i], 'meta': {'name': df_article['title'][i], 'url': df_article['url'][i]}})

In [ ]:
len(dicts_textContent)

In [ ]:
df_article.head(5)


In [ ]:
document_store.write_documents(dicts_textContent)


In [ ]:
from haystack.retriever.sparse import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2-covid", use_gpu=False)

In [ ]:
finder = Finder(reader, retriever)

## Ask questions

In [ ]:
prediction = finder.get_answers(question="Are children more susceptible to coronavirus?", top_k_retriever=5, top_k_reader=1)
print("****** \n")
print_answers(prediction, details="all")